In [1]:
%config IPCompleter.greedy=True

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [2]:
import import_ipynb
from functions import calc_vpc

ModuleNotFoundError: No module named 'import_ipynb'